In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import logging
from pathlib import Path
from typing import Any

import torchvision.transforms.v2.functional as v2F  # type: ignore
from torchvision import tv_tensors
from tqdm.auto import tqdm, trange

import bb
import bbedit
import tt
import tt_fcos

LOG = logging.getLogger(__name__)
tt.logging_init()

In [ ]:
CATEGORIES = [
    "chicken",
    "cow",
    "creeper",
    "enderman",
    "pig",
    "sheep",
    "skeleton",
    "spider",
    "zombie",
]
CATEGORIES.sort()

DATA_PATH = Path.home() / "data"

# Create dataset with set of images

In [ ]:
# Create a new dataset and copy in images
new = bb.MCDataset.new(
    dset_dir=DATA_PATH / "new",
    categories=CATEGORIES,
    input_images_dir=DATA_PATH / "minecraft-mobs/images",
)

# Run new dataset through latest model

In [ ]:
# Load model
model_path = DATA_PATH / "checkpoints/keep/best.pt"
trainer = tt_fcos.FCOSTrainer.load_checkpoint(model_path, project_dir=model_path.parent, score_thresh=0.5)

# Load dataset
new_path = DATA_PATH / "new"
new_dset = bb.MCDataset(new_path)

In [ ]:
# Run infer on images and display the results
bb.BBoxViewer(new_dset, infer_fn=trainer.infer).show_widget()

In [ ]:
# Run each image through the model and save the results in the dataset
for idx in trange(len(new_dset)):                                                                               
    image, target = new_dset[idx]                                                                                   
    det = trainer.infer(image)
    new_dset.add_annotation(idx, det)                                                                     
new_dset.rebuild_index()

In [ ]:
# Show targets saved in the dataset
new_dset.view()

In [ ]:
# Save model predictions
new_dset.save_annotations()

# Hand classify

In [ ]:
DATA_PATH = Path.home() / "data"
new_dset_path = DATA_PATH / "new"

In [ ]:
new_dset = bb.MCDataset(new_dset_path)
print(new_dset.image_info(5))
new_dset[5][1]

In [ ]:
x = new_dset[0][1]
x

In [ ]:
bbedit.DEBUG.clear_output()
bbedit.DEBUG

In [ ]:
bbe = bbedit.BBoxEdit(new_dset_path)
bbe.display()

In [ ]:
check_dset = bb.Dataset.load(new_dset_path)
check_dset.view()

# Stats

In [ ]:
data_dir = Path.home() / "data"
dset = bb.MCDataset(data_dir / "new")
# display(dset.dataset_stats())
# display(dset.category_stats())

df = dset.to_df()

In [ ]:
dset.dataset_stats()

In [ ]:
dset.category_stats()